In [ ]:
import base64
import json
import os
import requests

import urllib.parse

## Further reading
The authorization steps of this repo is from the guide in Notion: https://developers.notion.com/docs/authorization#authorizing-public-integrations
    
Detailed step-by-step intructions for this whole repo in this blog post: https://norahsakal.com/blog/create-public-notion-integration
        

## Notion public integration credentials
Note that your redirect_uri has to be **exactly** the same as the one you added when creating the integration in Notion

In [ ]:
oauth_client_id = "YOUR_OAUTH_CLIENT_ID"
oauth_client_secret = "YOUR_OAUTH_CLIENT_SECRET"
redirect_uri = "YOUR_REDIRECT_URL"

## Parse your redirect url

In [ ]:
parsed_redirect_uri = urllib.parse.quote_plus(redirect_uri)

## Create the authorization URL that authorizes your public integration access to users pages/databases
Visit the URL you received and authorize your app

In [ ]:
auth_url = f"https://api.notion.com/v1/oauth/authorize?owner=user&client_id={oauth_client_id}&redirect_uri={parsed_redirect_uri}&response_type=code"

# Visit the URL you received and authorize your Notion integration
print(auth_url)

## Save the URL you got redirected to after authorization

In [ ]:
redirect_url_response = "THE_URL_YOU_GOT_REDIRECTED_TO_AFTER_AUTHORIZATION"

## Extract and save the code from the redirect URL

In [ ]:
auth_code = redirect_url_response.split('code=')[-1].split('&state=')[0]
auth_code

## Base64 encode your credentials to exchange the code into an access token

In [ ]:
key_secret = f'{oauth_client_id}:{oauth_client_secret}'.encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

## Exchange the code for an access token

In [ ]:
base_url = 'https://api.notion.com/v1/oauth/token'

auth_headers = {
    'Authorization': f'Basic {b64_encoded_key}',
    'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8',
}

auth_data = {
    'grant_type': 'authorization_code',
    'code': auth_code,
    'redirect_uri':redirect_uri,
}

auth_resp = requests.post(base_url, headers=auth_headers, data=auth_data)
auth_resp.json()

## Save the access token for the user

Store all of the information your integration receives with the access token. You never know when your UI requirements or product requirements might change, and you find some use of this data. More info on Notion https://developers.notion.com/docs/authorization#user-grants-access

In [ ]:
access_token = auth_resp.json()['access_token']

## Search for which databases the user granted access to

In [ ]:
url = "https://api.notion.com/v1/search"

payload = {"page_size": 100}
headers = {
    "accept": "application/json",
    "Notion-Version": "2022-06-28",
    "content-type": "application/json",
    "authorization": f"Bearer {access_token}"
}

response = requests.post(url, json=payload, headers=headers)
response.json()

## Save the granted database data

In [ ]:
database_data = response.json()['results']
notion_database_id = database_data[0]['id']

## Create example data to add to database

In [ ]:
example_data = {
    "handle": "@SomeHandle",
    "tweet": "Here is a tweet"
}

## Add data to user table

In [ ]:
headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/json',
    'Notion-Version': '2021-08-16'
}

payload= {
    'parent': { 'database_id': notion_database_id },
    'properties': {
        'title': {
            'title': [
                {
                    'text': {
                        'content': example_data['handle']
                    }
                }
            ]
        },
        'tweet': {
            'rich_text':[
                {
                    'type':'text',
                    'text': {
                        'content': example_data['tweet']
                    }
                }
            ]
        }
    }
}

response = requests.post('https://api.notion.com/v1/pages', headers=headers,data=json.dumps(payload))
response.json()